# MODELOS PREDICTIVOS CON MACHINE/DEEP LEARNING

In [2]:
pip install xgboost scikit-learn pandas joblib scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 998.5 kB/s eta 0:00:000:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 MB 1.9 MB/s eta 0:00:0000:0100:03m
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Script robusto completo - listo para copiar/pegar
# Requisitos: pip install xgboost scikit-learn pandas joblib scipy

import os
import glob
import inspect
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
import joblib
from scipy import stats

# (opcional) callback import, puede fallar en versiones muy antiguas
try:
    from xgboost import callback as xgb_callback
except Exception:
    xgb_callback = None

RANDOM_STATE = 42

# ---------- CONFIG (ajusta aquí) ----------
FILE_PATH = "/home/jovyan/work/data/curated/"   # puede ser archivo (.parquet/.csv/.feather) o carpeta
TARGET_COL = 'NON_COMPLIANT_CONTRACT'   # si conoces el nombre ponlo aquí, si no lo detecta automáticamente
APPROVAL_THRESHOLD = 0.8
MODEL_OUTPUT = "xgb_loan_model_robust.joblib"
# -------------------------------------------

def load_table(path):
    """Carga un DataFrame desde path (archivo o carpeta). Soporta parquet/csv/feather."""
    
    return pd.read_parquet(path)
        

# 1) Cargar datos
df = load_table(FILE_PATH)
print("Datos cargados. Shape:", df.shape)

# 2) Preparar X e y (asegurar 0/1)
y = df[TARGET_COL].copy()

# convertir booleanos a 0/1
if y.dtype == "bool":
    y = y.astype(int)
else:
    # normalizar cadenas representando True/False/Yes/No
    sval = y.dropna().astype(str).str.lower().unique().tolist()
    sval_set = set(sval)
    if sval_set.issubset({"true","false","t","f","yes","no","y","n"}):
        map_true = {"true", "t", "yes", "y"}
        y = y.astype(str).str.lower().map(lambda x: 1 if x in map_true else 0)
    else:
        if pd.api.types.is_numeric_dtype(y):
            uniq = np.unique(y.dropna())
            if set(uniq).issubset({0,1}):
                y = y.astype(int)
            elif len(uniq) == 2:
                # mapear a 0/1 ordenado
                mapping = {uniq[0]: 0, uniq[1]: 1}
                y = y.map(mapping)
            else:
                raise ValueError(f"Target numérico no binario: {uniq}. Asigna TARGET_COL correcto.")
        else:
            uniq = y.dropna().unique()
            if len(uniq) == 2:
                mapping = {uniq[0]: 0, uniq[1]: 1}
                y = y.map(mapping)
            else:
                raise ValueError(f"Target categórico no binario: {uniq}. Asigna TARGET_COL correcto.")

# comprobar binariedad final
if not set(pd.Series(y.dropna()).unique()).issubset({0,1}):
    raise ValueError("Después de la conversión, la columna target no contiene sólo 0/1. Revisa TARGET_COL.")

X = df.drop(columns=[TARGET_COL]).copy()

# 4) Split train/test
stratify = y if len(np.unique(y)) > 1 else None
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=stratify)
print("Train/Test shapes:", X_train.shape, X_test.shape)

# 5) Detectar num y categ
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
# eliminar solapamientos
cat_cols = [c for c in cat_cols if c not in num_cols]

print(f"Numéricas: {len(num_cols)} / Categóricas: {len(cat_cols)}")

# 6) Preprocesamiento con fallback para OrdinalEncoder
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# OrdinalEncoder con fallback dependiendo la versión de sklearn
try:
    ord_enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    ord_enc_supports_use_encoded_value = True
except TypeError:
    # fallback: usar 'ignore' y luego reemplazar NaN por -1 tras transform
    ord_enc = OrdinalEncoder(handle_unknown="ignore")
    ord_enc_supports_use_encoded_value = False

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="__missing__")),
    ("ord", ord_enc)
])

transformers = []
if num_cols:
    transformers.append(("num", numeric_transformer, num_cols))
if cat_cols:
    transformers.append(("cat", categorical_transformer, cat_cols))

if not transformers:
    raise ValueError("No se detectaron columnas numéricas ni categóricas. Revisa tus features.")

preprocessor = ColumnTransformer(transformers=transformers, remainder="drop")

# 7) XGBoost base (n_jobs=1)
pos = np.sum(y_train == 1)
neg = np.sum(y_train == 0)
scale_pos_weight = 1.0
if pos > 0 and neg > 0:
    scale_pos_weight = float(neg) / float(pos)
    print(f"scale_pos_weight calculado: {scale_pos_weight:.3f}")

xgb = XGBClassifier(
    objective="binary:logistic",
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=RANDOM_STATE,
    n_jobs=1,
    tree_method="hist",
    scale_pos_weight=scale_pos_weight
)

pipe = Pipeline(steps=[("preproc", preprocessor), ("model", xgb)])

# 8) RandomizedSearchCV (n_jobs=1)
param_dist = {
    "model__n_estimators": [100, 200, 400],
    "model__max_depth": stats.randint(3, 8),
    "model__learning_rate": [0.01, 0.03, 0.05, 0.1],
    "model__subsample": [0.6, 0.8, 1.0],
    "model__colsample_bytree": [0.5, 0.7, 1.0],
    "model__gamma": [0, 0.1, 0.3]
}

rs = RandomizedSearchCV(pipe, param_distributions=param_dist, n_iter=12, cv=3,
                        scoring="roc_auc", random_state=RANDOM_STATE, verbose=2, n_jobs=1, refit=True)

# 9) Fit del RandomizedSearchCV (sin pasar eval_set en CV)
try:
    rs.fit(X_train, y_train)
except Exception as e:
    print("Advertencia: RandomizedSearchCV falló en el primer intento. Error:")
    print(e)
    print("Reintentando con n_iter=8")
    rs = RandomizedSearchCV(pipe, param_distributions=param_dist, n_iter=8, cv=3,
                            scoring="roc_auc", random_state=RANDOM_STATE, verbose=2, n_jobs=1, refit=True)
    rs.fit(X_train, y_train)

print("Mejores parámetros (RandomizedSearchCV):", rs.best_params_)
best_model = rs.best_estimator_

# 10) Re-entrenado final con early stopping en una partición de validación
X_tr_sub, X_val, y_tr_sub, y_val = train_test_split(
    X_train, y_train, test_size=0.15, random_state=RANDOM_STATE,
    stratify=y_train if len(np.unique(y_train))>1 else None
)

# obtener el preprocessor ya ajustado del pipeline (refit=True en RandomizedSearchCV asegura esto)
preproc = best_model.named_steps["preproc"]

# función segura para transformar y convertir NaN a -1 si usamos OrdinalEncoder(ignore)
def safe_transform(preproc_obj, X_df):
    arr = preproc_obj.transform(X_df)
    # reemplazar NaN por -1 (aplica tanto para categorías desconocidas como seguridad)
    # arr será numpy.array
    try:
        arr = np.where(np.isnan(arr), -1, arr)
    except Exception:
        # si no hay np.nan o no aplica, lo devolvemos tal cual
        pass
    return arr

# transformar conjuntos
try:
    X_tr_sub_t = safe_transform(preproc, X_tr_sub)
except Exception:
    # si preproc no está ajustado (caso raro), ajustar con X_tr_sub y transformar
    preproc = preproc.fit(X_tr_sub)
    X_tr_sub_t = safe_transform(preproc, X_tr_sub)

X_val_t = safe_transform(preproc, X_val)
X_test_t = safe_transform(preproc, X_test)

# extraer parámetros para XGB y crear nuevo XGB con esos hiperparámetros
best_params_for_xgb = {k.replace("model__",""): v for k, v in rs.best_params_.items() if k.startswith("model__")}

xgb_final = XGBClassifier(
    objective="binary:logistic",
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=RANDOM_STATE,
    n_jobs=1,
    tree_method="hist",
    scale_pos_weight=scale_pos_weight,
    **best_params_for_xgb
)

# -------------------- Entrenamiento con intentos para early stopping --------------------
trained_with_early_stopping = False
fit_sig = inspect.signature(xgb_final.fit)
fit_params_names = fit_sig.parameters.keys()

# Intento 1: early_stopping_rounds en la firma
if "early_stopping_rounds" in fit_params_names:
    try:
        xgb_final.fit(
            X_tr_sub_t,
            y_tr_sub,
            early_stopping_rounds=30,
            eval_set=[(X_val_t, y_val)],
            verbose=False
        )
        trained_with_early_stopping = True
        print("Entrenado con early_stopping_rounds (firma soportada).")
    except TypeError as e:
        print("TypeError usando early_stopping_rounds:", e)
        trained_with_early_stopping = False
    except Exception as e:
        print("Error usando early_stopping_rounds:", e)
        trained_with_early_stopping = False

# Intento 2: usar callbacks.EarlyStopping si está disponible y si fit acepta callbacks
if (not trained_with_early_stopping) and ("callbacks" in fit_params_names) and (xgb_callback is not None):
    try:
        # crear callback (try varios constructores)
        try:
            cb = xgb_callback.EarlyStopping(rounds=30, save_best=True, metric_name="logloss")
        except Exception:
            try:
                cb = xgb_callback.EarlyStopping(30)
            except Exception:
                cb = None
        if cb is not None:
            xgb_final.fit(
                X_tr_sub_t,
                y_tr_sub,
                eval_set=[(X_val_t, y_val)],
                callbacks=[cb],
                verbose=False
            )
            trained_with_early_stopping = True
            print("Entrenado con callbacks.EarlyStopping.")
    except Exception as e:
        print("No fue posible entrenar con callbacks.EarlyStopping:", e)
        trained_with_early_stopping = False

# Intento 3: fallback sin early stopping
if not trained_with_early_stopping:
    print("Advertencia: no se pudo usar early stopping. Entrenando SIN early stopping (fallback).")
    xgb_final.fit(X_tr_sub_t, y_tr_sub, verbose=False)

# -------------------- Fin entrenamiento --------------------

# 11) Predicción y métricas
y_prob_test = xgb_final.predict_proba(X_test_t)[:, 1]
y_pred_threshold = (y_prob_test >= APPROVAL_THRESHOLD).astype(int)

auc = roc_auc_score(y_test, y_prob_test)
print(f"\nROC AUC en test: {auc:.4f}")
print("\nReporte de clasificación con umbral (threshold = {:.4f}):".format(APPROVAL_THRESHOLD))
print(classification_report(y_test, y_pred_threshold))
cm = confusion_matrix(y_test, y_pred_threshold)
print("Matriz de confusión (verdadero x predicho):\n", cm)

# 12) Guardar resultados test
X_test_out = X_test.reset_index(drop=True).copy()
results = pd.DataFrame({
    "prob_pay": y_prob_test,
    "approve": y_pred_threshold,
    "actual": y_test.reset_index(drop=True)
})
output = pd.concat([X_test_out, results], axis=1)
output_file = "test_with_probs_robust.csv"
output.to_csv(output_file, index=False)
print(f"\nResultados de test guardados en: {output_file}")

# 13) Guardar pipeline final (preproc + modelo final)
final_pipeline = Pipeline(steps=[("preproc", preproc), ("model", xgb_final)])
joblib.dump(final_pipeline, MODEL_OUTPUT)
print(f"Pipeline final guardado en: {MODEL_OUTPUT}")

# 14) Top 10 imprimible
print("\nTop 10 clientes por probabilidad de pago (test set):")
top10 = output.sort_values("prob_pay", ascending=False).head(10)
print(top10.reset_index(drop=True).to_string(index=False))



Datos cargados. Shape: (322156, 49)
TARGET_COL detectado automáticamente: 'NON_COMPLIANT_CONTRACT'
Train/Test shapes: (257724, 48) (64432, 48)
Numéricas: 36 / Categóricas: 12
scale_pos_weight calculado: 11.280
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END model__colsample_bytree=1.0, model__gamma=0, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=100, model__subsample=0.6; total time=   7.0s
[CV] END model__colsample_bytree=1.0, model__gamma=0, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=100, model__subsample=0.6; total time=   6.4s
[CV] END model__colsample_bytree=1.0, model__gamma=0, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=100, model__subsample=0.6; total time=   6.3s
[CV] END model__colsample_bytree=1.0, model__gamma=0.1, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=400, model__subsample=0.6; total time=  12.7s
[CV] END model__colsample_bytree=1.0, model__gamma=0.1, mod

In [15]:
# 15) Top 1000 imprimible
print("\nTop 1000 clientes por probabilidad de pago (test set):")
top1000 = output.sort_values("prob_pay", ascending=False).head(1000)
print(top1000.reset_index(drop=True).to_string(index=False))


Top 1000 clientes por probabilidad de pago (test set):
   CLIENT_ID NAME_PRODUCT_TYPE GENDER  TOTAL_INCOME  AMOUNT_PRODUCT  INSTALLMENT             EDUCATION MARITAL_STATUS          HOME_SITUATION  REGION_SCORE  AGE_IN_YEARS  JOB_SENIORITY  HOME_SENIORITY  LAST_UPDATE OWN_INSURANCE_CAR  CAR_AGE  FAMILY_SIZE  REACTIVE_SCORING  PROACTIVE_SCORING  BEHAVIORAL_SCORING  DAYS_LAST_INFO_CHANGE  NUMBER_OF_PRODUCTS OCCUPATION  DIGITAL_CLIENT HOME_OWNER EMPLOYER_ORGANIZATION_TYPE  NUM_PREVIOUS_LOAN_APP  LOAN_ANNUITY_PAYMENT_SUM  LOAN_APPLICATION_AMOUNT_SUM  LOAN_CREDIT_GRANTED_SUM  NUM_STATUS_ANNULLED  NUM_STATUS_AUTHORIZED  NUM_STATUS_DENIED  NUM_STATUS_NOT_USED  NUM_FLAG_INSURED       DATE  CREDICT_CARD_BALANCE  CREDIT_CARD_LIMIT  CREDIT_CARD_PAYMENT  NUMBER_DRAWINGS_ATM  NUMBER_DRAWINGS  NUMBER_INSTALMENTS  KPI_TOTAL_SPEND  KPI_DEBT_RATIO KPI_AGE_GROUP  KPI_LOAN_VOLATILITY  KPI_APPROVAL_RATIO  KPI_DENIAL_RATE  prob_pay  approve  actual
ES182338770T         PRODUCT 1      M       1890.00      

In [1]:
# script simplificado listo para copiar/pegar
# Requisitos: pip install xgboost scikit-learn pandas joblib
import os
import glob
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
import joblib
from scipy import stats

RANDOM_STATE = 42

# ---------- CONFIG (ajusta si quieres) ----------
FILE_PATH = "/home/jovyan/work/data/curated/"
TARGET_COL = 'NON_COMPLIANT_CONTRACT'
APPROVAL_THRESHOLD = 0.8
MODEL_OUTPUT = "xgb_loan_model_simplified.joblib"
# -------------------------------------------

def load_table(path):
    return pd.read_parquet(path)
        
# 1) Cargar datos
df = load_table(FILE_PATH)
print("Datos cargados. Shape:", df.shape)

# 2) Preparar X e y (asegurar 0/1)
y = df[TARGET_COL].copy()

# Conversión sencilla y robusta a 0/1
if y.dtype == bool:
    y = y.astype(int)
elif pd.api.types.is_numeric_dtype(y):
    uniq = np.unique(y.dropna())
    if set(uniq).issubset({0,1}):
        y = y.astype(int)
    elif len(uniq) == 2:
        mapping = {uniq[0]: 0, uniq[1]: 1}
        y = y.map(mapping).astype(int)
    else:
        raise ValueError(f"Target numérico no binario: {uniq}. Asigna TARGET_COL correcto.")
else:
    uniq = pd.Series(y.dropna().astype(str).str.lower().unique())
    # casos claros true/false/yes/no
    tf_set = {"true","false","t","f","yes","no","y","n"}
    if set(uniq).issubset(tf_set):
        map_true = {"true","t","yes","y"}
        y = y.astype(str).str.lower().map(lambda x: 1 if x in map_true else 0)
    elif len(uniq) == 2:
        mapping = {uniq.iloc[0]: 0, uniq.iloc[1]: 1}
        y = y.astype(str).map(mapping).astype(int)
    else:
        raise ValueError(f"Target categórico no binario: {uniq.tolist()}. Asigna TARGET_COL correcto.")

# comprobación final
if not set(pd.Series(y.dropna()).unique()).issubset({0,1}):
    raise ValueError("Después de la conversión, la columna target no contiene sólo 0/1. Revisa TARGET_COL.")

X = df.drop(columns=[TARGET_COL]).copy()

# 3) Split train/test
stratify = y if len(np.unique(y)) > 1 else None
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=stratify)
print("Train/Test shapes:", X_train.shape, X_test.shape)

# 4) Detectar num y categ
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
cat_cols = [c for c in cat_cols if c not in num_cols]  # evitar solapamientos

print(f"Numéricas: {len(num_cols)} / Categóricas: {len(cat_cols)}")

if not num_cols and not cat_cols:
    raise ValueError("No se detectaron columnas numéricas ni categóricas. Revisa tus features.")

# 5) Preprocesamiento simple: imputer+scaler para num, imputer+onehot para cat
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# intentar compatibilidad para diferentes versiones de sklearn
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="__missing__")),
    ("onehot", ohe)
])

transformers = []
if num_cols:
    transformers.append(("num", numeric_transformer, num_cols))
if cat_cols:
    transformers.append(("cat", categorical_transformer, cat_cols))

preprocessor = ColumnTransformer(transformers=transformers, remainder="drop")

# 6) XGBoost base (scale_pos_weight calculado si aplica)
pos = np.sum(y_train == 1)
neg = np.sum(y_train == 0)
scale_pos_weight = float(neg) / float(pos) if (pos > 0 and neg > 0) else 1.0
print(f"scale_pos_weight: {scale_pos_weight:.3f}")

xgb = XGBClassifier(
    objective="binary:logistic",
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=RANDOM_STATE,
    n_jobs=1,
    tree_method="hist",
    scale_pos_weight=scale_pos_weight
)

pipe = Pipeline(steps=[("preproc", preprocessor), ("model", xgb)])

# 7) Búsqueda de hiperparámetros (RandomizedSearchCV)
param_dist = {
    "model__n_estimators": [100, 200, 400],
    "model__max_depth": stats.randint(3, 8),
    "model__learning_rate": [0.01, 0.03, 0.05, 0.1],
    "model__subsample": [0.6, 0.8, 1.0],
    "model__colsample_bytree": [0.5, 0.7, 1.0],
    "model__gamma": [0, 0.1, 0.3]
}

rs = RandomizedSearchCV(pipe, param_distributions=param_dist, n_iter=12, cv=3,
                        scoring="roc_auc", random_state=RANDOM_STATE, verbose=2, n_jobs=1, refit=True)

print("Iniciando RandomizedSearchCV (esto puede tardar según tamaño de datos y n_iter)...")
rs.fit(X_train, y_train)

print("Mejores parámetros (RandomizedSearchCV):", rs.best_params_)
best_pipeline = rs.best_estimator_  # ya está refiteado en todo X_train por RandomizedSearchCV

# 8) Predicción y métricas (usamos el pipeline completo: preproc + modelo)
y_prob_test = best_pipeline.predict_proba(X_test)[:, 1]
y_pred_threshold = (y_prob_test >= APPROVAL_THRESHOLD).astype(int)

auc = roc_auc_score(y_test, y_prob_test)
print(f"\nROC AUC en test: {auc:.4f}")
print("\nReporte de clasificación con umbral (threshold = {:.4f}):".format(APPROVAL_THRESHOLD))
print(classification_report(y_test, y_pred_threshold))
cm = confusion_matrix(y_test, y_pred_threshold)
print("Matriz de confusión (verdadero x predicho):\n", cm)

# 9) Guardar resultados test
X_test_out = X_test.reset_index(drop=True).copy()
results = pd.DataFrame({
    "prob_pay": y_prob_test,
    "approve": y_pred_threshold,
    "actual": y_test.reset_index(drop=True)
})
output = pd.concat([X_test_out, results], axis=1)
output_file = "test_with_probs_simplified.csv"
output.to_csv(output_file, index=False)
print(f"\nResultados de test guardados en: {output_file}")

# 10) Guardar pipeline final (preproc + modelo)
joblib.dump(best_pipeline, MODEL_OUTPUT)
print(f"Pipeline final guardado en: {MODEL_OUTPUT}")

# 11) Top 10 imprimible
print("\nTop 10 clientes por probabilidad (test set):")
top10 = output.sort_values("prob_pay", ascending=False).head(10)
print(top10.reset_index(drop=True).to_string(index=False))


Datos cargados. Shape: (322156, 49)
Train/Test shapes: (257724, 48) (64432, 48)
Numéricas: 36 / Categóricas: 12
scale_pos_weight: 11.280
Iniciando RandomizedSearchCV (esto puede tardar según tamaño de datos y n_iter)...
Fitting 3 folds for each of 12 candidates, totalling 36 fits


: 

: 

: 